In [ ]:
!git clone https://github.com/18401172024/SignLanguageDetection1.git

Cloning into 'SignLanguageDetection1'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), 3.13 MiB | 8.56 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
!ls

sample_data  SignLanguageDetection1


In [ ]:
!ls

sample_data  SignLanguageDetection1


In [ ]:
!pwd

/content


In [ ]:
%cd SignLanguageDetection1

/content/SignLanguageDetection1


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17739, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 17739 (delta 63), reused 36 (delta 36), pack-reused 17634 (from 3)
Receiving objects: 100% (17739/17739), 17.12 MiB | 21.83 MiB/s, done.
Resolving deltas: 100% (12053/12053), done.


In [ ]:
%cd yolov5

/content/SignLanguageDetection1/yolov5


In [ ]:
!pip install -qr requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.1 MB/s eta 0:00:00


In [ ]:
import torch

In [ ]:
from IPython.display import Image, clear_output
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 2.8.0+cu126 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15095MB, multi_processor_count=40, uuid=a696f649-3dc9-54ec-9c3e-8da482e604c0, pci_bus_id=0, pci_device_id=4, pci_domain_id=0, L2_cache_size=4MB)


In [ ]:
! pwd

/content/SignLanguageDetection1/yolov5


In [ ]:
%cd /content

/content


In [31]:
!curl -L "https://github.com/18401172024/SignLanguageDetection1/raw/refs/heads/main/yolov5-demo.v2i.yolov5pytorch.zip" > SignData.zip; unzip SignData.zip; rm SignData.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3314k  100 3314k    0     0  2185k      0  0:00:01  0:00:01 --:--:-- 4168k
Archive:  SignData.zip
replace data.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [30]:
%cat /content/SignLanguageDetection1/data.yaml

cat: /content/SignLanguageDetection1/data.yaml: No such file or directory


In [ ]:
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
num_classes

'6'

In [ ]:
%cat /content/yolov5/models/yolov5s.yaml

cat: /content/yolov5/models/yolov5s.yaml: No such file or directory


In [ ]:
from IPython.core.magic import register_line_cell_magic
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/SignLanguageDetection1/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
%cd /content/SignLanguageDetection1

/content/SignLanguageDetection1


In [ ]:
%%time
%cd /content/SignLanguageDetection1/yolov5/

/content/SignLanguageDetection1/yolov5
CPU times: user 1.43 ms, sys: 65 µs, total: 1.49 ms
Wall time: 1.33 ms


In [34]:
!python train.py --img 416 --batch 16 --epochs 100 --data /content/SignLanguageDetection1/data.yaml --cfg ./models/custom_yolov5s.yaml --weights yolov5s.pt --name yolov5s_results

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-11-21 08:16:53.862918: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763713013.884601    6243 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763713013.890904    6243 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763713013.906101    6243 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763713013.906130    6243 computation_placer.cc:177] computation placer already registere

/content/SignLanguageDetection1/yolov5/data.yaml
/content/data.yaml
